# Tasteful Recommender System Tests

Complete testing of the recommendation system: search tool, agent state, and full integration.

In [ ]:
import os
import json
import time
from tools import WebSearchTool
from agent_state import AgentState
from recommender import AutonomousTastefulRecommender

## Environment Check

In [ ]:
# Check required API keys
brave_key = os.getenv("BRAVE_API_KEY")
openai_key = os.getenv("OPENAI_API_KEY")

print("🔑 API KEYS STATUS")
print("=" * 30)
print(f"BRAVE_API_KEY:  {'✅ Set' if brave_key else '❌ Missing'} {brave_key[:8] + '...' if brave_key else ''}")
print(f"OPENAI_API_KEY: {'✅ Set' if openai_key else '❌ Missing'} {openai_key[:8] + '...' if openai_key else ''}")

can_search = bool(brave_key)
can_recommend = bool(brave_key and openai_key)

print(f"\n🔍 Can test search: {'✅ Yes' if can_search else '❌ No'}")
print(f"🤖 Can test full system: {'✅ Yes' if can_recommend else '❌ No'}")

if not brave_key:
    print("\n📝 Get Brave API key: https://brave.com/search/api/")
if not openai_key:
    print("📝 Set OpenAI API key in environment")

## WebSearchTool Tests

In [ ]:
# Test search tool
search_tool = WebSearchTool()

print("🔍 WEBSEARCHTOOL TESTS")
print("=" * 30)
print(f"Tool name: {search_tool.name()}")
print(f"Description: {search_tool.description()}")

if can_search:
    # Test basic search
    query = "best coffee shops Seattle 2024"
    print(f"\nTesting: '{query}'")
    
    result = search_tool.execute(query=query, max_results=5)
    parsed = json.loads(result)
    
    print(f"Status: {parsed['status']}")
    if parsed['status'] == 'success':
        print(f"Results: {parsed['count']}")
        print("\nFirst result:")
        if parsed['results']:
            r = parsed['results'][0]
            print(f"  Title: {r['title']}")
            print(f"  URL: {r['url']}")
            print(f"  Snippet: {r['snippet'][:80]}...")
        print("\n✅ Search working")
    else:
        print(f"❌ Search failed: {parsed['error']}")
else:
    print("⏭️ Skipping search tests - no BRAVE_API_KEY")

In [ ]:
# Test multiple queries
if can_search:
    test_queries = [
        "best budget laptops 2024",
        "top rated hiking backpacks",
        "mechanical keyboards programming"
    ]
    
    print("🔍 MULTIPLE QUERY TESTS")
    print("=" * 30)
    
    for i, query in enumerate(test_queries, 1):
        result = search_tool.execute(query=query, max_results=3)
        parsed = json.loads(result)
        
        status_icon = "✅" if parsed['status'] == 'success' else "❌"
        count = parsed.get('count', 0)
        print(f"{i}. {status_icon} '{query}' → {count} results")
    
    print("\n✅ Multiple queries working")
else:
    print("⏭️ Skipping multiple query tests")

In [ ]:
# Test error handling
print("🔍 ERROR HANDLING TESTS")
print("=" * 30)

# Test with no API key
original_key = os.environ.get("BRAVE_API_KEY")
if original_key:
    os.environ["BRAVE_API_KEY"] = ""  # Temporarily remove

try:
    no_key_tool = WebSearchTool()
    result = no_key_tool.execute(query="test")
    parsed = json.loads(result)
    
    if parsed['status'] == 'error' and 'not found' in parsed['error']:
        print("✅ Handles missing API key correctly")
    else:
        print("❌ API key error handling failed")
finally:
    if original_key:
        os.environ["BRAVE_API_KEY"] = original_key  # Restore

# Test empty query
if can_search:
    empty_result = search_tool.execute(query="")
    empty_parsed = json.loads(empty_result)
    print(f"Empty query status: {empty_parsed['status']}")

print("\n✅ Error handling working")

## AgentState Tests

In [ ]:
# Test agent state
print("🧠 AGENTSTATE TESTS")
print("=" * 25)

# Create and test state
state = AgentState()
print(f"Initial state - Query: '{state.original_query}'")
print(f"Research data: {len(state.research_data)} items")
print(f"Reasoning steps: {len(state.reasoning_steps)}")
print(f"Completed: {state.completed}")

# Test adding data
state.original_query = "Test query for backpacks"
state.add_reasoning_step("Need to search for backpack options")
state.add_reasoning_step("Should look at reviews and prices")

# Add mock search data
mock_search_result = json.dumps({
    "query": "best backpacks",
    "status": "success",
    "results": [{"title": "Test", "url": "test.com", "snippet": "Test snippet"}]
})
state.add_research_data("backpack_search", mock_search_result)

# Add recommendations
state.final_recommendations = [
    {"name": "Test Backpack 1", "description": "Great for hiking"},
    {"name": "Test Backpack 2", "description": "Budget option"}
]
state.completed = True

print(f"\nAfter updates:")
print(f"Query: {state.original_query}")
print(f"Steps: {len(state.reasoning_steps)}")
print(f"Research: {list(state.research_data.keys())}")
print(f"Recommendations: {len(state.final_recommendations)}")
print(f"Completed: {state.completed}")

print("\n✅ AgentState working correctly")

## Recommender Integration Tests

In [ ]:
# Test recommender setup
print("🤖 RECOMMENDER INTEGRATION TESTS")
print("=" * 40)

if can_recommend:
    recommender = AutonomousTastefulRecommender()
    
    print(f"Available tools: {list(recommender.tools.keys())}")
    print(f"Max iterations: {recommender.max_iterations}")
    print(f"State type: {type(recommender.state).__name__}")
    
    # Test tool execution through recommender
    print("\nTesting tool execution:")
    tool_result = recommender._execute_tool("web_search", {
        "query": "best gaming mouse under $50",
        "max_results": 3
    })
    
    tool_parsed = json.loads(tool_result)
    print(f"Tool test status: {tool_parsed['status']}")
    print(f"Tool test results: {tool_parsed.get('count', 0)}")
    
    if tool_parsed['status'] == 'success':
        print("✅ Tool integration working")
    else:
        print(f"❌ Tool integration failed: {tool_parsed['error']}")
        
    # Test tools description
    tools_desc = recommender._get_tools_description()
    print(f"\nTools description: {tools_desc}")
    
else:
    print("⏭️ Skipping integration tests - missing API keys")

## Performance Tests

In [ ]:
# Test search performance
if can_search:
    print("⚡ PERFORMANCE TESTS")
    print("=" * 25)
    
    # Time search operations
    test_query = "best wireless headphones 2024"
    times = []
    
    for i in range(3):
        start = time.time()
        result = search_tool.execute(query=test_query, max_results=5)
        end = time.time()
        
        parsed = json.loads(result)
        if parsed['status'] == 'success':
            times.append(end - start)
            print(f"Run {i+1}: {times[-1]:.2f}s ({parsed['count']} results)")
    
    if times:
        avg_time = sum(times) / len(times)
        print(f"\nAverage search time: {avg_time:.2f}s")
        
        if avg_time < 3.0:
            print("✅ Good performance (< 3s)")
        else:
            print("⚠️ Slow performance (> 3s)")
    else:
        print("❌ No successful searches for timing")
else:
    print("⏭️ Skipping performance tests")

## End-to-End Recommendation Test

In [ ]:
# Full system test - uncomment to run
# WARNING: Uses API calls and costs money

if can_recommend:
    print("🎯 END-TO-END TEST READY")
    print("=" * 30)
    print("Uncomment the code below to run full recommendation test")
    print("This will make API calls to both Brave and OpenAI")
    print()
    
    # UNCOMMENT TO RUN FULL TEST:
    # test_query = "I need a good mechanical keyboard for programming under $120"
    # print(f"Query: {test_query}")
    # print("=" * 50)
    # 
    # recommender = AutonomousTastefulRecommender()
    # recommender.max_iterations = 3  # Limit for testing
    # 
    # result = recommender.process_query(test_query)
    # recommender.print_results()
    # 
    # print(f"\n📊 SUMMARY:")
    # print(f"Iterations: {result['steps']}")
    # print(f"Recommendations: {len(result['recommendations'])}")
    # print(f"Searches performed: {len(recommender.state.research_data)}")
    # 
    # for search_key in recommender.state.research_data.keys():
    #     print(f"  - {search_key}")
    
else:
    print("Cannot run end-to-end test - missing API keys")

## System Summary

In [ ]:
print("📋 SYSTEM TEST SUMMARY")
print("=" * 40)

# Component status
components = [
    ("WebSearchTool", "✅" if can_search else "❌"),
    ("AgentState", "✅"),  # Always works
    ("Recommender", "✅" if can_recommend else "❌"),
    ("Integration", "✅" if can_recommend else "❌")
]

for component, status in components:
    print(f"{status} {component}")

print(f"\n🔧 Setup Status:")
print(f"   API Keys: {2 if can_recommend else 1 if can_search else 0}/2")
print(f"   Search: {'Ready' if can_search else 'Need BRAVE_API_KEY'}")
print(f"   Full System: {'Ready' if can_recommend else 'Need OPENAI_API_KEY'}")

print(f"\n📈 Architecture:")
print(f"   Files: 4 core + 1 test notebook")
print(f"   Tools: 1 (WebSearchTool)")
print(f"   Dependencies: Brave Search API")
print(f"   Complexity: Minimal")

if can_recommend:
    print("\n🚀 System ready for recommendations!")
elif can_search:
    print("\n⚠️ Set OPENAI_API_KEY for full functionality")
else:
    print("\n⚠️ Set BRAVE_API_KEY to begin testing")